In [1]:
{-# LANGUAGE RecordWildCards #-}

In [2]:
import Data.Function (on)
import Data.Map.Strict (Map)
import Data.Monoid (Monoid(..))
import Data.Set (Set)

import qualified Data.Map.Strict as M
import qualified Data.Set as S

In [3]:
data Graph v e =
  Graph
  {
    allVertices   :: Set v
  , allEdges      :: Set e
  , incomingEdges :: Map v (Set (v, e))
  , outgoingEdges :: Map v (Set (v, e))
  }
    deriving (Eq, Ord, Read, Show)

In [4]:
instance (Ord v, Ord e) => Monoid (Graph v e) where
  mempty =
    Graph
    {
      allVertices   = S.empty
    , allEdges      = S.empty
    , incomingEdges = M.empty
    , outgoingEdges = M.empty
    }
  mappend x y =
    Graph
    {
      allVertices   = on S.union allVertices x y
    , allEdges      = on S.union allEdges x y
    , incomingEdges = on (M.unionWith S.union) incomingEdges x y
    , outgoingEdges = on (M.unionWith S.union) outgoingEdges x y
    }

In [5]:
data LabeledWeight l w =
  LabeledWeight
  {
    label  :: l
  , weight :: w
  }
    deriving (Eq, Ord, Read, Show)

In [6]:
data Vertex a =
    SuperSource
  | SuperSink
  | Vertex a
    deriving (Eq, Ord, Read, Show)

In [7]:
data Edge a =
    Edge a
  | ReversedEdge a
    deriving (Eq, Ord, Read, Show)

In [12]:
addEdge :: (Ord v, Ord e) => Graph v e -> (v, v, e) -> Graph v e
addEdge Graph{..} (from, to, edge) =
  Graph
  {
    allVertices   = S.insert from $ S.insert to allVertices
  , allEdges      = S.insert edge allEdges
  , incomingEdges = M.insertWith S.union to   (S.singleton edge) incomingEdges
  , outgoingEdges = M.insertWith S.union from (S.singleton edge) outgoingEdges
  }

In [14]:
makeGraph :: (Ord v, Ord e) => [(v, v, e)] -> Graph v e
makeGraph = foldl addEdge mempty